In [1]:
# Importing the libraries

import numpy as np 
import pandas as pd
from tqdm import tqdm
import math
import matplotlib.pylab as plt
from scipy import sparse
from scipy.linalg import sqrtm
import pandas as pd
import numpy as np
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
complete_train = pd.read_csv('train.csv')
complete_train_og = complete_train.copy()
test = pd.read_csv('kaggle_baseline.csv')

In [3]:
df2 = complete_train
def set_top_scores(group):
    # Find the highest rating
    highest_rating = group['rating'].max()
    if highest_rating == 5:
        group['new_rating'] = group['rating'].apply(lambda x: 1 if x == highest_rating or x == highest_rating - 1 else 0)
    else:
        group['new_rating'] = group['rating'].apply(lambda x: 1 if x == highest_rating else 0)        
    
    return group

df2 = df2.groupby('user_id').apply(set_top_scores).reset_index(drop=True)
df2 = df2.drop(['rating'], axis=1)  # Remove original rating column
df2.rename(columns={'new_rating': 'rating'}, inplace=True)
df2 = df2.drop(['age', 'release_date', 'sex'], axis=1)
df2

/var/folders/yb/mm0hrmlx3g9dv10c735753m00000gn/T/ipykernel_70621/2786937693.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df2 = df2.groupby('user_id').apply(set_top_scores).reset_index(drop=True)


,user_id,title,movie_id,rating
0,1,Hercules (1997),1566,1
1,1,Erin Brockovich (2000),3408,1
2,1,Toy Story (1995),1,1
3,1,Titanic (1997),1721,1
4,1,One Flew Over the Cuckoo's Nest (1975),1193,1
...,...,...,...,...
800162,6040,"Paris, Texas (1984)",1305,1
800163,6040,Breaking Away (1979),3359,1
800164,6040,"Simple Plan, A (1998)",2391,1
800165,6040,"Haunted World of Edward D. Wood Jr., The (1995)",722,0


In [4]:
df = complete_train
def set_top_scores(group):
    # Find the highest rating
    highest_rating = group['rating'].max()
    if highest_rating == 5 or highest_rating == 4:
        group['new_rating'] = group['rating'].apply(lambda x: 1 if x == highest_rating or x == highest_rating - 1 else 0)
    else:
        group['new_rating'] = group['rating'].apply(lambda x: 1 if x == highest_rating else 0)        
    
    return group

df = df.groupby('user_id').apply(set_top_scores).reset_index(drop=True)
df = df.drop(['rating'], axis=1)  # Remove original rating column
df.rename(columns={'new_rating': 'rating'}, inplace=True)
df = df.drop(['age', 'release_date', 'sex'], axis=1)
df

/var/folders/yb/mm0hrmlx3g9dv10c735753m00000gn/T/ipykernel_70621/1299700687.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('user_id').apply(set_top_scores).reset_index(drop=True)


,user_id,title,movie_id,rating
0,1,Hercules (1997),1566,1
1,1,Erin Brockovich (2000),3408,1
2,1,Toy Story (1995),1,1
3,1,Titanic (1997),1721,1
4,1,One Flew Over the Cuckoo's Nest (1975),1193,1
...,...,...,...,...
800162,6040,"Paris, Texas (1984)",1305,1
800163,6040,Breaking Away (1979),3359,1
800164,6040,"Simple Plan, A (1998)",2391,1
800165,6040,"Haunted World of Edward D. Wood Jr., The (1995)",722,0


## Eigene Umwandlung

In [5]:
# movie_title_lookup = df[['movie_id', 'title']].drop_duplicates().set_index('movie_id')['title'].to_dict()

# uids = df['user_id'].unique()
# mids = df['movie_id'].unique()
# um_pairs = pd.DataFrame([(uid, mid) for uid in uids for mid in mids], columns=['user_id', 'movie_id'])

# um_pairs = um_pairs.merge(df[['user_id', 'movie_id', 'rating']], on=['user_id', 'movie_id'], how='left')
# um_pairs['title'] = um_pairs['movie_id'].map(movie_title_lookup).fillna(0)
# um_pairs['rating'].fillna(0, inplace=True)
# df = um_pairs
# um_pairs






/var/folders/yb/mm0hrmlx3g9dv10c735753m00000gn/T/ipykernel_68321/3033812230.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  um_pairs['rating'].fillna(0, inplace=True)


,user_id,movie_id,rating,title
0,1,1566,1.0,Hercules (1997)
1,1,3408,1.0,Erin Brockovich (2000)
2,1,1,1.0,Toy Story (1995)
3,1,1721,1.0,Titanic (1997)
4,1,1193,1.0,One Flew Over the Cuckoo's Nest (1975)
...,...,...,...,...
22227195,6040,2703,0.0,Broken Vessels (1998)
22227196,6040,2845,0.0,White Boys (1999)
22227197,6040,3607,0.0,One Little Indian (1973)
22227198,6040,1360,0.0,Identification of a Woman (Identificazione di ...


## Standard Data Prep

In [5]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(df, test_size=0.01, random_state=7)

x_train = train[["user_id", "movie_id"]].values
y_train = train["rating"].values

x_val = val[["user_id", "movie_id"]].values
y_val = val["rating"].values

In [6]:
from sklearn.model_selection import train_test_split

train2, val2 = train_test_split(df2, test_size=0.01, random_state=7)

x_train2 = train2[["user_id", "movie_id"]].values
y_train2 = train2["rating"].values

x_val2 = val2[["user_id", "movie_id"]].values
y_val2 = val2["rating"].values

In [7]:
user_ids = df["user_id"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
movie_ids = df["movie_id"].unique().tolist()
movie2movie_encoded = {x: i for i, x in enumerate(movie_ids)}
movie_encoded2movie = {i: x for i, x in enumerate(movie_ids)}
df["user"] = df["user_id"].map(user2user_encoded)
df["movie"] = df["movie_id"].map(movie2movie_encoded)

num_users = len(user2user_encoded)
num_movies = len(movie_encoded2movie)
df["rating"] = df["rating"].values.astype(np.float32)
# min and max ratings will be used to normalize the ratings later
min_rating = min(df["rating"])
max_rating = max(df["rating"])

print(
    "Number of users: {}, Number of Movies: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_movies, min_rating, max_rating
    )
)

Number of users: 6040, Number of Movies: 3680, Min rating: 0.0, Max rating: 1.0


## Utility

In [9]:
def plot_train_val_losses(history):
    plt.plot(history.history["loss"])
    plt.plot(history.history["val_loss"])
    plt.title("model loss")
    plt.ylabel("loss")
    plt.xlabel("epoch")
    plt.legend(["train", "test"], loc="upper left")
    plt.axis([0,len(history.history["loss"]),np.min(history.history["loss"]),np.max(history.history["val_loss"])])
    plt.show()




def evaluate(predict_f,data_train,data_test):
    """ RMSE-based predictive performance evaluation with pandas. """
    ids_to_estimate = zip(data_test.user_id, data_test.movie_id)
    list_users = set(data_train.user_id)
    estimated = np.array([predict_f(u,i) if u in list_users else 3 for (u,i) in ids_to_estimate ])
    real = data_test.rating.values
    return compute_rmse(estimated, real)

def compute_rmse(y_pred, y_true):
    """ Compute Root Mean Squared Error. """
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

def precision(recommended_items, relevant_items):
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)
    
    return precision_score

def recall(recommended_items, relevant_items):  
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]
    
    return recall_score

def AP(recommended_items, relevant_items):
   
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))
    ap_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return ap_score

def evaluate_algorithm_top(test, recommender_object, at=25, thr_relevant = 0.85):
    
    cumulative_precision = 0.0
    cumulative_recall = 0.0
    cumulative_AP = 0.0
    
    num_eval = 0


    for user_id in tqdm(test.user_id.unique()):
        
        relevant_items = test[test.user_id==user_id]
        thr = np.quantile(relevant_items.rating,thr_relevant)
        relevant_items = np.array(relevant_items[relevant_items.rating >=thr].movie_id.values)
        if len(relevant_items)>0:
            
            recommended_items = top_recomendations(recommender_object,user_id, at=at)
            num_eval+=1

            cumulative_precision += precision(recommended_items, relevant_items)
            cumulative_recall += recall(recommended_items, relevant_items)
            cumulative_AP += AP(recommended_items, relevant_items)
            
    cumulative_precision /= num_eval
    cumulative_recall /= num_eval
    MAP = cumulative_AP / num_eval
    
    print("Recommender results are: Precision = {:.4f}, Recall = {:.4f}, MAP = {:.4f}".format(
        cumulative_precision, cumulative_recall, MAP)) 

## Attempt 1

In [10]:
EMBEDDING_SIZE = 10

class RecommenderNetV(keras.Model):
    def __init__(self, num_users, num_movies, embedding_size, **kwargs):
        super(RecommenderNetV, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_movies = num_movies
        self.embedding_size = embedding_size
        self.user_movie_embedding = layers.Embedding(
            num_users+num_movies,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.user_bias = layers.Embedding(num_users, 1)
        self.movie_bias = layers.Embedding(num_movies, 1)

    def call(self, inputs):
        user_vector = self.user_movie_embedding(inputs[:, 0])
        movie_vector = self.user_movie_embedding(inputs[:, 1]+num_users)
        user_bias = self.user_bias(inputs[:, 0])
        movie_bias = self.movie_bias(inputs[:, 1])
        dot_user_movie = tf.tensordot(user_vector, movie_vector, 2)
        # Add all the components (including bias)
        x = dot_user_movie + user_bias + movie_bias
        return x


class LogisticRecommenderNet(keras.Model):
    def __init__(self, num_users, num_movies, embedding_size, **kwargs):
        super(LogisticRecommenderNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_movies = num_movies
        self.embedding_size = embedding_size
        # Embeddings for users and movies
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.movie_embedding = layers.Embedding(
            num_movies,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        # Biases for users and movies
        self.user_bias = layers.Embedding(num_users, 1)
        self.movie_bias = layers.Embedding(num_movies, 1)

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        movie_vector = self.movie_embedding(inputs[:, 1])
        user_bias = self.user_bias(inputs[:, 0])
        movie_bias = self.movie_bias(inputs[:, 1])
        # Dot product of user and movie vectors + user and movie biases
        dot_user_movie = tf.reduce_sum(user_vector * movie_vector, axis=1, keepdims=True)
        x = dot_user_movie + user_bias + movie_bias
        # Apply sigmoid activation to model probability
        return tf.nn.sigmoid(x)

In [11]:
def top_recomendations(model_object, user_id,at = 25):
    movies_watched_by_user = df[df.user_id == user_id]

    movies_not_watched = df[df["movie_id"].isin(movies_watched_by_user.movie_id.values)]["movie_id"]
    movies_not_watched = list(set(movies_not_watched).intersection(set(movie2movie_encoded.keys())))
    movies_not_watched = [[movie2movie_encoded.get(x)] for x in movies_not_watched]

    
    user_encoder = user2user_encoded.get(user_id)
    user_movie_array = np.hstack(([[user_encoder]] * len(movies_not_watched), movies_not_watched))

    ratings = model_object.predict(user_movie_array,verbose=0).flatten()
    top_ratings_indices = ratings.argsort()[-at:][::-1]
    recommended_movie_ids = [movie_encoded2movie.get(movies_not_watched[x][0]) for x in top_ratings_indices]
    return recommended_movie_ids, ratings[top_ratings_indices]

In [41]:
mf_model_start = RecommenderNetV(num_users, num_movies, EMBEDDING_SIZE)
mf_model_start.compile(
    loss=tf.keras.losses.MeanSquaredError(), 
    optimizer=keras.optimizers.Adam(learning_rate=0.0005)
)
mf_history = mf_model_start.fit(
    x=x_train,
    y=y_train,
    batch_size=128,
    epochs=10,
    verbose=1,
    validation_data=(x_val, y_val),
)
evaluate_algorithm_top(val, mf_model_start, at = 25)

Epoch 1/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 887us/step - loss: 0.3071 - val_loss: 0.1914
Epoch 2/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 5s 865us/step - loss: 0.1895 - val_loss: 0.1889
Epoch 3/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 964us/step - loss: 0.1871 - val_loss: 0.1891
Epoch 4/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 935us/step - loss: 0.1877 - val_loss: 0.1909
Epoch 5/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - loss: 0.1880 - val_loss: 0.1905
Epoch 6/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 885us/step - loss: 0.1880 - val_loss: 0.1919
Epoch 7/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 904us/step - loss: 0.1886 - val_loss: 0.1914
Epoch 8/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 5s 801us/step - loss: 0.1882 - val_loss: 0.1924
Epoch 9/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 5s 804us/step - loss: 0.1889 - val_loss: 0.1902
Epoch 10/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 5s 806us/step - loss: 0.1889 - val_loss: 0.1914


100%|██████████| 3340/3340 [02:49<00:00, 19.68it/s]

Recommender results are: Precision = 0.0147, Recall = 0.2746, MAP = 0.0444


In [42]:
mf_model_start1 = RecommenderNetV(num_users, num_movies, EMBEDDING_SIZE)
mf_model_start1.compile(
    loss=tf.keras.losses.MeanSquaredError(), 
    optimizer=keras.optimizers.Adam(learning_rate=0.001)
)
mf_history = mf_model_start1.fit(
    x=x_train,
    y=y_train,
    batch_size=128,
    epochs=20,
    verbose=1,
    validation_data=(x_val, y_val),
)
evaluate_algorithm_top(val, mf_model_start1, at = 25)

Epoch 1/20
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 907us/step - loss: 0.2653 - val_loss: 0.2069
Epoch 2/20
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 988us/step - loss: 0.2095 - val_loss: 0.2147
Epoch 3/20
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 962us/step - loss: 0.2184 - val_loss: 0.2141
Epoch 4/20
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 978us/step - loss: 0.2260 - val_loss: 0.2210
Epoch 5/20
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 967us/step - loss: 0.2302 - val_loss: 0.2451
Epoch 6/20
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 951us/step - loss: 0.2363 - val_loss: 0.2390
Epoch 7/20
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 0.2349 - val_loss: 0.2326
Epoch 8/20
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 5s 861us/step - loss: 0.2383 - val_loss: 0.2253
Epoch 9/20
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 971us/step - loss: 0.2395 - val_loss: 0.2261
Epoch 10/20
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 895us/step - loss: 0.2431 - val_loss: 0.2517
Epoch 11/20
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 934us/step - loss: 0.2399 - val_loss: 0.2483
Epoch 12/2

100%|██████████| 3340/3340 [02:50<00:00, 19.58it/s]

Recommender results are: Precision = 0.0149, Recall = 0.2757, MAP = 0.0479


In [43]:
mf_model_start2 = RecommenderNetV(num_users, num_movies, EMBEDDING_SIZE)
mf_model_start.compile(
    loss=tf.keras.losses.MeanSquaredError(), 
    optimizer=keras.optimizers.Adam(learning_rate=0.0005)
)
mf_history = mf_model_start.fit(
    x=x_train,
    y=y_train,
    batch_size=128,
    epochs=10,
    verbose=1,
    validation_data=(x_val, y_val),
)
#evaluate_algorithm_top(val, mf_model_start, at = 25)

Epoch 1/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 835us/step - loss: 0.1880 - val_loss: 0.1917
Epoch 2/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 957us/step - loss: 0.1880 - val_loss: 0.1911
Epoch 3/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 929us/step - loss: 0.1886 - val_loss: 0.1917
Epoch 4/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 907us/step - loss: 0.1885 - val_loss: 0.1934
Epoch 5/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 5s 819us/step - loss: 0.1889 - val_loss: 0.1912
Epoch 6/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 5s 810us/step - loss: 0.1889 - val_loss: 0.1919
Epoch 7/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 961us/step - loss: 0.1884 - val_loss: 0.1923
Epoch 8/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 0.1889 - val_loss: 0.1916
Epoch 9/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 0.1885 - val_loss: 0.1935
Epoch 10/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 972us/step - loss: 0.1888 - val_loss: 0.1909


In [45]:
evaluate_algorithm_top(val, mf_model_start2, at = 25)

100%|██████████| 3340/3340 [02:50<00:00, 19.58it/s]

Recommender results are: Precision = 0.0153, Recall = 0.2865, MAP = 0.0451


In [44]:
mf_model_new = LogisticRecommenderNet(num_users, num_movies, EMBEDDING_SIZE)
mf_model_new.compile(
    loss=tf.keras.losses.MeanSquaredError(), 
    optimizer=keras.optimizers.Adam(learning_rate=0.0005)
)
mf_history = mf_model_new.fit(
    x=x_train,
    y=y_train,
    batch_size=128,
    epochs=10,
    verbose=1,
    validation_data=(x_val, y_val),
)
evaluate_algorithm_top(val, mf_model_new, at = 25)

Epoch 1/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 5s 688us/step - loss: 0.2358 - val_loss: 0.1999
Epoch 2/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 4s 676us/step - loss: 0.1944 - val_loss: 0.1896
Epoch 3/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 4s 713us/step - loss: 0.1857 - val_loss: 0.1867
Epoch 4/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 5s 796us/step - loss: 0.1823 - val_loss: 0.1853
Epoch 5/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 5s 787us/step - loss: 0.1802 - val_loss: 0.1844
Epoch 6/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 4s 688us/step - loss: 0.1782 - val_loss: 0.1837
Epoch 7/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 4s 683us/step - loss: 0.1772 - val_loss: 0.1833
Epoch 8/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 5s 846us/step - loss: 0.1758 - val_loss: 0.1830
Epoch 9/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 4s 689us/step - loss: 0.1747 - val_loss: 0.1826
Epoch 10/10
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 934us/step - loss: 0.1735 - val_loss: 0.1822


100%|██████████| 3340/3340 [02:57<00:00, 18.79it/s]

Recommender results are: Precision = 0.0146, Recall = 0.2717, MAP = 0.0452


In [9]:
mf_model_new = LogisticRecommenderNet(num_users, num_movies, EMBEDDING_SIZE)
mf_model_new.compile(
    loss=tf.keras.losses.MeanSquaredError(), 
    optimizer=keras.optimizers.Adam(learning_rate=0.0005)
)
mf_history = mf_model_new.fit(
    x=x_train,
    y=y_train,
    batch_size=128,
    epochs= 17,
    verbose=1,
    validation_data=(x_val, y_val),
)
evaluate_algorithm_top(val, mf_model_new, at = 25)

Epoch 1/17
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 5s 686us/step - loss: 0.2363 - val_loss: 0.2007
Epoch 2/17
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 4s 684us/step - loss: 0.1950 - val_loss: 0.1892
Epoch 3/17
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 5s 871us/step - loss: 0.1852 - val_loss: 0.1863
Epoch 4/17
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 5s 848us/step - loss: 0.1821 - val_loss: 0.1850
Epoch 5/17
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 908us/step - loss: 0.1797 - val_loss: 0.1840
Epoch 6/17
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 5s 788us/step - loss: 0.1785 - val_loss: 0.1836
Epoch 7/17
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 5s 766us/step - loss: 0.1769 - val_loss: 0.1833
Epoch 8/17
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 4s 703us/step - loss: 0.1755 - val_loss: 0.1830
Epoch 9/17
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 0.1746 - val_loss: 0.1827
Epoch 10/17
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - loss: 0.1739 - val_loss: 0.1824
Epoch 11/17
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 5s 767us/step - loss: 0.1727 - val_loss: 0.1821
Epoch 12/17


100%|██████████| 3340/3340 [03:03<00:00, 18.15it/s]

Recommender results are: Precision = 0.0147, Recall = 0.2747, MAP = 0.0464


In [25]:
mf_model_new3 = LogisticRecommenderNet(num_users, num_movies, EMBEDDING_SIZE)
mf_model_new3.compile(
    loss=tf.keras.losses.MeanSquaredError(), 
    optimizer=keras.optimizers.Adam(learning_rate=0.0001)
)
mf_history = mf_model_new3.fit(
    x=x_train,
    y=y_train,
    batch_size=128,
    epochs=60,
    verbose=1,
    validation_data=(x_val, y_val),
)
#evaluate_algorithm_top(val, mf_model_new3, at = 25)

Epoch 1/60
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 5s 699us/step - loss: 0.2471 - val_loss: 0.2379
Epoch 2/60
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 5s 880us/step - loss: 0.2353 - val_loss: 0.2265
Epoch 3/60
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 5s 782us/step - loss: 0.2237 - val_loss: 0.2157
Epoch 4/60
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 4s 711us/step - loss: 0.2129 - val_loss: 0.2072
Epoch 5/60
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 5s 743us/step - loss: 0.2044 - val_loss: 0.2011
Epoch 6/60
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 4s 652us/step - loss: 0.1984 - val_loss: 0.1969
Epoch 7/60
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - loss: 0.1938 - val_loss: 0.1940
Epoch 8/60
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 5s 752us/step - loss: 0.1908 - val_loss: 0.1920
Epoch 9/60
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 5s 730us/step - loss: 0.1884 - val_loss: 0.1905
Epoch 10/60
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 4s 638us/step - loss: 0.1863 - val_loss: 0.1894
Epoch 11/60
6189/6189 ━━━━━━━━━━━━━━━━━━━━ 4s 721us/step - loss: 0.1851 - val_loss: 0.1886
Epoch 12/6

In [26]:
evaluate_algorithm_top(val, mf_model_new3, at = 25)
print(top_recomendations(mf_model_new3, 1000, at = 25))

  0%|          | 0/3340 [00:00<?, ?it/s]

100%|██████████| 3340/3340 [03:04<00:00, 18.12it/s]

Recommender results are: Precision = 0.0074, Recall = 0.2733, MAP = 0.0457
([596, 380, 2973, 377, 1201, 2085, 594, 1029, 1023, 2080, 2700, 2194, 2137, 2087, 2804, 3702, 2366, 2353, 1287, 1104, 595, 1954, 2687, 1233, 3153], array([0.91154355, 0.86413103, 0.8373187 , 0.7916293 , 0.7250679 ,
       0.70799524, 0.701083  , 0.5668616 , 0.5660668 , 0.5440038 ,
       0.53373253, 0.5200685 , 0.51977426, 0.51509374, 0.4999377 ,
       0.48826393, 0.47459283, 0.45716706, 0.45703134, 0.44794562,
       0.44706926, 0.42869732, 0.41613147, 0.4010609 , 0.38597077],
      dtype=float32))


2024-03-29 16:34:56.042181: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at strided_slice_op.cc:117 : INVALID_ARGUMENT: slice index 1 of dimension 1 out of bounds.
2024-03-29 16:34:56.042199: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: slice index 1 of dimension 1 out of bounds.
	 [[{{node logistic_recommender_net_1_1/strided_slice_1}}]]


InvalidArgumentError: Graph execution error:

Detected at node logistic_recommender_net_1_1/strided_slice_1 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/Users/max/Library/Python/3.11/lib/python/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/max/Library/Python/3.11/lib/python/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/Users/max/Library/Python/3.11/lib/python/site-packages/ipykernel/kernelapp.py", line 701, in start

  File "/Users/max/Library/Python/3.11/lib/python/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/opt/homebrew/Cellar/python@3.11/3.11.6_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/opt/homebrew/Cellar/python@3.11/3.11.6_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/base_events.py

In [19]:
from tqdm.std import tqdm
import csv
import os
os.environ["TQDM_CONSOLE"] = "True"

with open('finalAttempt2.csv', 'w',encoding='UTF8') as f:
    # create the csv writer
    writer = csv.writer(f)
    # write a row to the csv file
    writer.writerow(['user_id', 'prediction'])
    pbar = tqdm(total=len(test.user_id.unique()))
    for user_id in tqdm(test.user_id.unique(),leave=False):
        #n_ratings = ready_df[ready_df['uid']==user_id].movie_id.count()
        #if(n_ratings > 30):
        relevant_items = top_recomendations(mf_model, user_id,at = 25)
        #else:
        # relevant_items = topPopular.predict_top(user_id, at=25)
        list_relevants = ' '.join([str(elem) for elem in relevant_items])
        writer.writerow([str(user_id),list_relevants])
        pbar.update(1)

  0%|          | 0/6037 [00:00<?, ?it/s]

100%|██████████| 6037/6037 [3:15:24<00:00,  2.03s/it]  

In [ ]:
evaluate_algorithm_top(val, mf_model, at = 25)

  5%|▌         | 314/6040 [10:11<3:18:20,  2.08s/it]

## Dieser Github Attempt

In [13]:
train = train.copy()
val = val.copy()

y_train = train['rating']
train.drop(['rating'], axis=1, inplace=True)

train['user_id'] = train['user_id'].astype('str')
train['movie_id'] = train['movie_id'].astype('str')

y_val = val["rating"]
val.drop(['rating'], axis=1, inplace=True)
val['user_id'] = val['user_id'].astype('str')
val['movie_id'] = val['movie_id'].astype('str')

test['user_id'] = test['user_id'].astype('str')
test['movie_id'] = test['movie_id'].astype('str')

KeyError: 'movie_id'

## Notizen

- Normale Ratings, 1 Epoche, Batch_size 128: 0.9 (RMSE) 0.0933 (MAP)


- Normale Rating, mit Implicit, 4 E, Batch 128: 0.4752 val los, Kaggle 0.00182